In [18]:
# Using Colomoto to convert format
# colomoto-docker --bind .
import biolqm
from rpy2.robjects.packages import importr
boolnet = importr("BoolNet")
import pandas as pd
import numpy as np
import libsbml


# Build SBML-qual model from rules in spreadsheet

In [19]:
model_name = "Selvaggio2020"
txt_file = f"Sources/{model_name}.txt"
sbml_file_generated = f"Sources/{model_name}_boolnet.sbml"
xlsx_file = f"Sources/{model_name}.xlsx"
sbml_file = f"Sources/{model_name}_constructed.sbml"

## xlsx to txt (boolnet)

In [36]:
# Excel file with the rules
df = pd.read_excel(xlsx_file, sheet_name='Logical_rules')

with open(txt_file, 'w') as f:
    f.write('targets, factors\n')
    for index, row in df.iterrows():
        if row['Rules'] is not np.nan:
            f.write(f"{row['Node']}, {row['Rules']}\n")

## txt (boolnet) to SBML-qual

In [38]:
# Load the network in text file
net = boolnet.loadNetwork(txt_file)

# Export to SBML
boolnet.toSBML(net, sbml_file_generated)
print(boolnet.loadSBML(sbml_file_generated))


Boolean network with 59 genes

Involved genes:
EGFR HGFR ITG_AB FAK_SRC FAK_SRC_2 RAS RAF1 MEK ERK PI3K ILK AKT GSK3B DVL CK1 FAT4 LATS NOTCH YAP_TAZ RPTP RAP1 TGFBR JAK BCat NFkB SMAD STAT3 JNK HIF1a PAK CSL TCF_LEF SNAIL SLUG ZEB miR200 ECad ECad_AJ ECad_AJ_2 BCat_AJ p120_AJ AJ AJ_2 FA FA_2 FA_3 EGF ITGab HGF ECM ROS_Hypoxia cMET WNT FAT4L DELTA RPTPL p120Ctn_ADJ TGFB IL6

Transition functions:
EGFR = ((EGF | (FAK_SRC_2 & ITGab) | TGFBR) & !RPTP)
HGFR = ((HGF | ITGab) & !RPTP)
ITG_AB = (ECM & RAP1)
FAK_SRC = ((ROS_Hypoxia & !ITGab) | ((EGFR | cMET) & !ITGab) | (ITGab & !(ROS_Hypoxia | (EGFR & cMET))))
FAK_SRC_2 = ((ROS_Hypoxia & ITGab) | (ITGab & cMET & EGFR))
RAS = (EGFR | FAK_SRC | JAK | cMET | TGFBR)
RAF1 = RAS
MEK = RAF1
ERK = MEK
PI3K = (RAS | FAK_SRC)
ILK = (ITGab & PI3K)
AKT = (PI3K & ILK)
GSK3B = (!AKT & !ERK & !DVL)
DVL = (CK1 & YAP_TAZ)
CK1 = WNT
FAT4 = (FAT4L & !ERK)
LATS = ((FAT4 | (ECad_AJ & BCat_AJ)) & !PI3K)
NOTCH = DELTA
YAP_TAZ = !LATS
RPTP = (RPTPL & !FAK_SRC_2 & !R

## Add comments from Excel file to transition notes

In [39]:
reader = libsbml.SBMLReader()
document = reader.readSBML(sbml_file_generated)
model = document.getModel().getPlugin('qual')

# Add comments from Excel file to transition notes
for index, row in df.iterrows():
    if row['Comments'] is not np.nan:
        transition_id = f"tr_{row['Node']}"
        transition = model.getTransition(transition_id)
        if transition is not None:
            transition.setNotes(row['Comments'], True)

# Export to SBML
libsbml.writeSBMLToFile(document, sbml_file)

1

# Proper indenting

In [1]:
from lxml import etree

tree = etree.parse("Sources/Selvaggio2020_ginsim.sbml")

# Pretty print with proper indentation
pretty_xml = etree.tostring(tree, pretty_print=True, encoding='utf-8').decode('utf-8')

with open("Sources/Selvaggio2020_ginsim_formatted.sbml", "w") as f:
    f.write(pretty_xml)